In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as pyo
import numpy as np
import os
from tqdm.notebook import tqdm

In [2]:
file_lenght=-len(os.path.abspath(".").split("\\")[-1])
csv_path =  os.path.abspath(".")[:file_lenght]+"Data\\"

In [4]:
myData = pd.read_csv(csv_path+"Updated_Data_2.csv",header=0,engine='c',index_col=[0],low_memory=False)
myData.head()

E:\user\anaconda\envs\Forecasting_sales\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,ITINERARY_CODE,ITINERARY_NAME,PARTNER_CODE,PARTNERCATEGORY_CODE,PARTNERCATEGORYTYPE_CODE,VENDOR,SECTOR CODE,SECTOR NAME,REGION CODE,REGION NAME,...,UNITPRICE,SOLD QUANTITY,TURNOVER,DISCOUNT,GREGORIAN_DATE,HOLIDAY,SCOLAR_HOLIDAY,ISLAM_EVENT,TEMPERATURE,HUMIDITY
0,S02001,"Samedi, Centre ville 1",02/0002,M,M,R0201,201,EURL ADEMLAIT-CHLEF,2,CHLEF,...,88.000000,12.0,1056.000000,0.0,2017-11-25,0,0,None,55.59,0.54
1,S02001,"Samedi, Centre ville 1",02/0004,GM,GM,R0201,201,EURL ADEMLAIT-CHLEF,2,CHLEF,...,83.333333,36.0,2999.999988,0.0,2017-11-25,0,0,None,55.59,0.54
2,S02001,"Samedi, Centre ville 1",02/0004,GM,GM,R0201,201,EURL ADEMLAIT-CHLEF,2,CHLEF,...,88.000000,36.0,3168.000000,0.0,2017-11-25,0,0,None,55.59,0.54
3,S02001,"Samedi, Centre ville 1",02/0005,M,M,R0201,201,EURL ADEMLAIT-CHLEF,2,CHLEF,...,45.000000,12.0,540.000000,0.0,2017-11-25,0,0,None,55.59,0.54
4,S02001,"Samedi, Centre ville 1",02/0006,M,M,R0201,201,EURL ADEMLAIT-CHLEF,2,CHLEF,...,45.000000,24.0,1080.000000,0.0,2017-11-25,0,0,None,55.59,0.54


<h4>Detail about our Data and optimize the reading process for quick interactions</h4>

In [14]:
myData.info() #first look

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15692870 entries, 0 to 15692869
Data columns (total 23 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ITINERARY_CODE            object 
 1   ITINERARY_NAME            object 
 2   PARTNER_CODE              object 
 3   PARTNERCATEGORY_CODE      object 
 4   PARTNERCATEGORYTYPE_CODE  object 
 5   VENDOR                    object 
 6   SECTOR CODE               int64  
 7   SECTOR NAME               object 
 8   REGION CODE               int64  
 9   REGION NAME               object 
 10  PRODUCT_CODE              object 
 11  PRODUCT_NAME              object 
 12  PRODUCTPAGE_CODE          object 
 13  UNITPRICE                 float64
 14  SOLD QUANTITY             float64
 15  TURNOVER                  float64
 16  DISCOUNT                  float64
 17  GREGORIAN_DATE            object 
 18  HOLIDAY                   int64  
 19  SCOLAR_HOLIDAY            int64  
 20  ISLAM_EVENT           

In [15]:
myData.info(memory_usage='deep') #deep look

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15692870 entries, 0 to 15692869
Data columns (total 23 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ITINERARY_CODE            object 
 1   ITINERARY_NAME            object 
 2   PARTNER_CODE              object 
 3   PARTNERCATEGORY_CODE      object 
 4   PARTNERCATEGORYTYPE_CODE  object 
 5   VENDOR                    object 
 6   SECTOR CODE               int64  
 7   SECTOR NAME               object 
 8   REGION CODE               int64  
 9   REGION NAME               object 
 10  PRODUCT_CODE              object 
 11  PRODUCT_NAME              object 
 12  PRODUCTPAGE_CODE          object 
 13  UNITPRICE                 float64
 14  SOLD QUANTITY             float64
 15  TURNOVER                  float64
 16  DISCOUNT                  float64
 17  GREGORIAN_DATE            object 
 18  HOLIDAY                   int64  
 19  SCOLAR_HOLIDAY            int64  
 20  ISLAM_EVENT           

In [ ]:
for c in myData:
    k=myData[str(c)].unique()
    if len(k)<60 and str(c): #our data is composed of 15 million rows (category of 60 element is still saving memory usage)
        myData[str(c)]=myData[str(c)].astype("category")

In [17]:
myData.info(memory_usage='deep') #result

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15692870 entries, 0 to 15692869
Data columns (total 23 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   ITINERARY_CODE            object  
 1   ITINERARY_NAME            object  
 2   PARTNER_CODE              object  
 3   PARTNERCATEGORY_CODE      category
 4   PARTNERCATEGORYTYPE_CODE  category
 5   VENDOR                    object  
 6   SECTOR CODE               int64   
 7   SECTOR NAME               object  
 8   REGION CODE               category
 9   REGION NAME               category
 10  PRODUCT_CODE              category
 11  PRODUCT_NAME              category
 12  PRODUCTPAGE_CODE          category
 13  UNITPRICE                 category
 14  SOLD QUANTITY             float64 
 15  TURNOVER                  float64 
 16  DISCOUNT                  float64 
 17  GREGORIAN_DATE            object  
 18  HOLIDAY                   category
 19  SCOLAR_HOLIDAY            category
 20  

<p>We saved <b>7.0 GB</b> of memory</p>

<h2>Time series plot</h2>
<ul>
    <li>Daily Revenue all clients & all sectors (chiffre affaire par jour)</li>
    <li>Daily Revenue all product per sector (MCA par jour de tout produit par secteur)</li>
    <li>Daily Revenue all clients per Product range (MCA par jour selon la gamme produit de tout clients)</li>
</ul>

### Parameters as Daily-Monthly Progress, Menu Gamme Produit, Secteur-Region

<h4>Daily Revenue all clients & all sectors (chiffre affaire par jour)</h4>

In [3]:
cols=["SECTOR NAME","REGION NAME","PRODUCT_NAME","PRODUCTPAGE_CODE","TURNOVER","DISCOUNT","GREGORIAN_DATE"]
types={"SECTOR NAME": "category","REGION NAME": "category","PRODUCT_NAME": "category","PRODUCTPAGE_CODE": "category"}
myData = pd.read_csv(csv_path+"Updated_Data_2.csv",header=0,engine='c',usecols=cols,low_memory=False)
myData.head()

,SECTOR NAME,REGION NAME,PRODUCT_NAME,PRODUCTPAGE_CODE,TURNOVER,DISCOUNT,GREGORIAN_DATE
0,EURL ADEMLAIT-CHLEF,CHLEF,ENTIER,Laits Blancs,1056.000000,0.0,2017-11-25
1,EURL ADEMLAIT-CHLEF,CHLEF,DE 1L,Laits Blancs,2999.999988,0.0,2017-11-25
2,EURL ADEMLAIT-CHLEF,CHLEF,VIVA,Laits Blancs,3168.000000,0.0,2017-11-25
3,EURL ADEMLAIT-CHLEF,CHLEF,DE 50CL,Laits Blancs,540.000000,0.0,2017-11-25
4,EURL ADEMLAIT-CHLEF,CHLEF,DE 50CL,Laits Blancs,1080.000000,0.0,2017-11-25


In [5]:
myData.info(memory_usage='deep') #deep look

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15692870 entries, 0 to 15692869
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   SECTOR NAME       object 
 1   REGION NAME       object 
 2   PRODUCT_NAME      object 
 3   PRODUCTPAGE_CODE  object 
 4   TURNOVER          object 
 5   DISCOUNT          float64
 6   GREGORIAN_DATE    object 
dtypes: float64(1), object(6)
memory usage: 6.1 GB


In [4]:
import cufflinks as cf               # third-party library for Pandas 
import plotly.graph_objs as go
%matplotlib inline
cf.go_offline()
pyo.init_notebook_mode(connected=True)
import ipywidgets as widgets
import itertools

#### Extract Attributes

In [5]:
sectors = myData[["SECTOR NAME","REGION NAME"]].drop_duplicates()
sectorDict = { region.upper() : [sector for sector in sectors[sectors["REGION NAME"]==region]["SECTOR NAME"]] for region in sectors["REGION NAME"].drop_duplicates().tolist()}
sectorDict

{'CHLEF': ['EURL ADEMLAIT-CHLEF'],
 'LAGHOUAT': ['DOUIHASNI AYACHE'],
 'OUM EL BOUAGHI': ['BOUCHAKOUR RAFIK'],
 'BATNA': ['BAAZIZI MOHAMED FOUZI', 'BAAZIZI MOHAMED FOUZI BARIKA'],
 'BEJAIA': ['MD CHERIF HAKIM', 'FOURAR AHCENE', 'IBRIR MERZOUK'],
 'BISKRA': ['EURL GOUBAA DISTRIBUTION'],
 'BECHAR': ['LAAMA MOUSSA'],
 'BLIDA': ['HADDAD ZAHIR'],
 'ALGER': ['MD CHERIF SAMIR',
  'IBRIR SLIMANE',
  'IBRIR ABDELHAK',
  'EURL MILK SODA FOOD',
  'IBRIR LARBI ALGER',
  'EURL CANDILAIT'],
 'BOUIRA': ['IBRIR ABDESLAM'],
 'TEBESSA': ['EURL NASR ECHARK'],
 'TLEMCEN': ['IBRIRKHELIL'],
 'TIARET': ['IBRIR SAMIR'],
 'TIZI OUZOU': ['SARL CANDY DISTRIBUTION', 'SARL CANDY DISTRIBUTION 2 -TO'],
 'DJELFA': ['BOUZIDI ABDELKRIM'],
 'JIJEL': ['SARL JGOAA'],
 'SETIF': ['EURL CANDY MILK', 'EURL DELICIOUS MILK'],
 'SAIDA': ['BELHOCINE Abdellatif'],
 'SKIKDA': ['HAMAZA LAKHDAR', 'FOURAR ALI'],
 'SIDI BEL ABBES': ['EURL EL TEWFIK'],
 'ANNABA': ['EURL ILYES FOOD', 'ANNABA 2'],
 'GUELMA': ['AZERINE YACINE', 'TABOU HAMI

In [6]:
productGamme = myData[["PRODUCT_NAME","PRODUCTPAGE_CODE"]].drop_duplicates()
productDict = { gamme.lower() : [product.lower() for product in productGamme[productGamme["PRODUCTPAGE_CODE"]==gamme]["PRODUCT_NAME"]] for gamme in productGamme["PRODUCTPAGE_CODE"].drop_duplicates().tolist()}
productDict

{'laits blancs': ['entier',
  'de 1l',
  'viva',
  'de 50cl',
  'silhouette',
  'de 1l sans lactose'],
 'laits aromatisés': ['choco 20cl',
  'choco 125 ml',
  'fraise 125 ml',
  'candy caramel 125 ml',
  'choco 1l',
  'fraise 20cl',
  'banane 125 ml'],
 'laits & jus': ['twist o-a 20cl',
  'twist o-f-b 20cl',
  'twist o-f-b 1l',
  'twist o-a 1l',
  'twist o-m 20cl',
  'twist p-a 20cl',
  'twist o-m 1l',
  'twist p-a 1l',
  'twist o-f-b 125ml',
  'twist o-a 125ml'],
 'boissons': ['cocktail 1l',
  'orange 1l',
  'citronnade',
  'nectar de grenade',
  'cocktail 20cl',
  'orange 20cl'],
 'le maitre cuisinier': ['preparation culinaire'],
 'le maitre glacier': ['ice cream - vanille',
  'ice cream - chocolat',
  'ice cream - fraise'],
 'poudre etui': ['etui 500g']}

In [79]:
%%timeit
# myData[myData["GREGORIAN_DATE"].str.match("2018-01")]
myData["GREGORIAN_DATE"].str.slice(stop=7).drop_duplicates().sort_values()

4.32 s ± 99.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [80]:
%%timeit
myData["GREGORIAN_DATE"].map(lambda x: x[:7]).drop_duplicates().sort_values()

3.89 s ± 45.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [82]:
%%timeit
myData["GREGORIAN_DATE"].drop_duplicates().str.slice(stop=7).drop_duplicates().sort_values()

223 ms ± 4.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [81]:
%%timeit
myData["GREGORIAN_DATE"].drop_duplicates().map(lambda x: x[:7]).drop_duplicates().sort_values()

217 ms ± 4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
NewDataStock = myData
NewDataStock["GREGORIAN_MONTH"] = myData["GREGORIAN_DATE"].map(lambda x: x[:7])
NewDataStock.head(1)

,SECTOR NAME,REGION NAME,PRODUCT_NAME,PRODUCTPAGE_CODE,TURNOVER,DISCOUNT,GREGORIAN_DATE,GREGORIAN_MONTH
0,EURL ADEMLAIT-CHLEF,CHLEF,ENTIER,Laits Blancs,1056.0,0.0,2017-11-25,2017-11


In [14]:
layout = go.Layout(
                    title="Serie temporelles - Ventes de produits laitiers",
                    xaxis= {"title": "Time"},
                    yaxis= {"title": "per 1M dinar"}
                    )
def DataDisp(Mode,productGamme,regionSlider):
    
    data = []
     
    for gamme,region in tqdm(list(itertools.product(productGamme,regionSlider))):
        y = NewDataStock 
        if gamme != 'tous':
            y= y[y["PRODUCTPAGE_CODE"].str.lower().str.match(gamme)]
        if region != 'TOUS':
            y= y[y["REGION NAME"].str.upper().str.match(region)]

        if Mode=='Monthly':
            x= NewDataStock["GREGORIAN_MONTH"].drop_duplicates().sort_values()
            y= y.groupby("GREGORIAN_MONTH")["TURNOVER"].sum()/1000000
        else:
            x= NewDataStock["GREGORIAN_DATE"].drop_duplicates().sort_values()
            y= y.groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000

        data.append(
            go.Scatter(
                x = x,
                y = y,
                mode="lines",
                name=f'PRD:{gamme}, REG:{region}, {Mode}',
            )
        )
    fig = go.Figure(data=data,layout=layout)
    pyo.iplot(fig)

In [16]:
Mode = widgets.RadioButtons(
    options=['Daily', 'Monthly'],
    value='Daily', # Defaults to 'pineapple'
    layout={'width': 'max-content'}, # If the items' names are long
    description='Mode:',
    disabled=False
)

productGamme = widgets.SelectMultiple(
    options=['tous']+list(productDict.keys()),
    value=['tous'],
    rows=4,
    description='Gamme',
    disabled=False
)

regionSlider = widgets.SelectMultiple(
    options=['TOUS']+list(sectorDict.keys()),
    value=['TOUS'],
    rows=4,
    description='Region',
    disabled=False
)

def TestDisp(Mode,productGamme,regionSlider):
    print("gamme:")
    for g in productGamme:
        print(g,end=", ")
    print("\nregion:")
    for g in regionSlider:
        print(g,end=", ")
        
widgets.interactive(DataDisp,Mode=Mode,productGamme=productGamme,regionSlider=regionSlider)

interactive(children=(RadioButtons(description='Mode:', layout=Layout(width='max-content'), options=('Daily', …

In [75]:
widgets.interactive(TestDisp,Mode=Mode,productGamme=productGamme,regionSlider=regionSlider)

interactive(children=(RadioButtons(description='Mode:', layout=Layout(width='max-content'), options=('Daily', …